In [3]:
import json, time, os, pathlib
import rocksdb
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import clear_output, display

API_BASE = os.getenv("DS_API_BASE", "http://localhost:8080")
ROCKS_PATH = os.getenv("ROCKS_PATH", "tmpdb")
SESSION_ID = os.getenv("DEMO_SESSION_ID", "session-1")


ModuleNotFoundError: No module named 'rocksdb'

In [4]:
db = rocksdb.DB(ROCKS_PATH, rocksdb.Options(create_if_missing=True))


NameError: name 'rocksdb' is not defined

In [ ]:
import requests

def ds_chat(text: str):
    payload = {"text": text}
    r = requests.post(f"{API_BASE}/v1/chat", json=payload, timeout=30)
    r.raise_for_status()
    return r.json()


In [ ]:
def write_to_ledger(prompt, answer):
    key = f"p-{int(time.time()*1000)}".encode()
    confidence = {
        "dxp_service_definition": "partial",
        "dxp_user_centric": "low",
        "evidence_level": "operational/anecdotal",
        "p_value_suggested": "P50"
    }
    record = {
        "prompt": prompt,
        "answer": answer,
        "ts": time.time(),
        "confidence": confidence
    }
    db.put(key, json.dumps(record).encode())
    return key


In [ ]:
def draw_tree(session_id=SESSION_ID):
    clear_output(wait=True)
    g = nx.DiGraph()
    g.add_node(session_id)
    it = db.iterkeys()
    it.seek_to_first()
    for k in it:
        k_dec = k.decode()
        g.add_node(k_dec)
        g.add_edge(session_id, k_dec)
    plt.figure(figsize=(7, 5))
    nx.draw(g, with_labels=True, node_color="lightblue", node_size=1300, font_size=7)
    plt.show()


In [ ]:
def send_and_show(prompt):
    resp = ds_chat(prompt)
    answer = resp.get("text") or resp.get("answer") or str(resp)
    key = write_to_ledger(prompt, answer)
    print("LLM:", answer)
    draw_tree()
    return key


In [ ]:
def search_ledger(query: str):
    it = db.iteritems()
    it.seek_to_first()
    results = []
    for k, v in it:
        data = json.loads(v.decode())
        blob = (data.get("prompt", "") + " " + data.get("answer", "")).lower()
        if query.lower() in blob:
            results.append((k.decode(), data))
    return results


In [ ]:
def get_all_confidence():
    it = db.iteritems()
    it.seek_to_first()
    rows = []
    for k, v in it:
        data = json.loads(v.decode())
        if "confidence" in data:
            rows.append((k.decode(), data["confidence"]))
    return rows
